This is part of the publication on automated detection of downy mildew on leaf discs.

**High-throughput phenotyping of leaf discs infected with grapevine downy mildew using trained convolutional neural networks**\
Zendler D, Nagarjun M, Schwandner A, Hausmann L, Zyprian E

Please be aware that further reading is required to get reasonable results. The code presented here was used to generated the CNN models for the above mentioned publication.

## 1. Load all the things:

In [1]:
from pathlib import Path
from pathlib import PurePath
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import RMSprop
from keras.optimizers import Adam
from keras.optimizers import Nadam
from keras.optimizers import Adadelta
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

## 2. Define all the variables needed

First off we will define a few parameters that will help us for the downstream code:
  - dropout = this will be adjusted in case out CNN is overfitting
  - learning_rate = this will influence big the weights adjustments will be
  - batch_size = the number of images that is analyzed in a batch
  - train_dir = path to the training data
  - val_dir = path to the validation data
  - img_size = the pixel resolution of the image slices
  - nr_train_img = the total number of training images (calss1 + class2)
  - nr_val_img = the total number of validation images (class1 + class2)
  - nr_epoch = the number of epochs for training the CNN

In [3]:
# Change this to something between 0.2 and 0.5
dropout = 0.5

# For RMSprop, Adam, Nadam the learning rate should be around 0.001 - 0.0001
# For Adadelta the learning rate has to be set to 0.1
# For SGD the learning rate should be set to 0.01
learning_rate = 0.0001

# Increase or dcrease this number according to your system
batch_size = 16

# The directory in which we stored the training and validation data for the two different classes.
# !!! Make sure to adjust this !!!
train_dir = 'data4_update/train/'
val_dir = 'data4_update/validation/'

# Image resolution in pixels for the image slices
x = 100 #height
y = 119 #width
img_size = (x, y, 3)

# Number of training and validation images
nr_train_img = 1936 # don't forget
nr_val_img = 874  # don't forget

# Number of training epochs; for testing 5 for final training 15 - 30
nr_epoch = 30

## 3. Set up the CNN

Let's generate the CNN:
  - we chose 3 four convolution layers with 32, 32, 64 and 128 nodes
  - each layer has the ReLU activation function
  - each layer is pooled by 2 x 2 (see MaxPooling2D for more information)
  - Two dense layers with each 256 inputs

Please be aware that you might have to change the architecture of your network according to your data. Also the used optimizer is a thing of trail and error.

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_size), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), kernel_initializer='he_normal'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Nadam(lr=learning_rate),
              metrics=['accuracy'])

## 4. Create image data generators and image data augmentation

Next we will generate a ImageDataGenerator. This is a helpful tool when the number of images is limited.
The ImageDataGenerator will perform socalled image augmentation with out images. This means the images will changed slightly according to the supplied options. This will hopefully give the model more flexibility e.g. in terms of differing light conditions.\
This will be done only for the training images as we want to validate the model with not augmented data.


Further with the option rescale we change the values of the RGB picture from training and validation that ranges from 1 to 255 to values between 0 and 1. This is needed for tensorflow.



In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,              # Rescaling the RGB values to a number between 0 and 1
        shear_range=0.2,             # Shear angle in counter-clockwise direction in degrees
        zoom_range=0.2,              # Range for random zoom
        horizontal_flip=True,        # Random horizontal flipping of the image
        vertical_flip=True)          # Random vertical flipping of the image
#        brightness_range=[0.5,1.5])  # Randomly adjusting the brightness
#        width_shift_range=0.2,
#        height_shift_range=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

### 4.1 Have a look at the image data augmentation

Next we will visualize the image augmentation options from above to get an impression what is going to be done with the images and if that makes sense.


Please specify an image from one of the classes in the training directory. Idealy you want to see something like this:


![img_augmentation](img_augmen.png)


(If there are more than 9 images in the training folder after testing the augmentation this will throw an error.)

In [ ]:
img_path = 'data4_update/train/spo/LDA_Plate1_I_s06.jpg_10_17.png' # Specify a image from your training data to visualize the image augmentation preview

try:
    img = load_img(img_path)
except:
    print("No file path supplied")
else:
    img = load_img(img_path)
    img_array = img_to_array(img)
    img_array_reshape = img_array.reshape((1,) + img_array.shape)  
    i = 0
    for batch in train_datagen.flow(img_array_reshape, batch_size=1,
                                    save_to_dir=train_dir, 
                                    save_prefix='augmentation_preview', 
                                    save_format='jpeg'):
        i += 1
        if i > 8:
            break  # otherwise the generator would loop indefinitely

    data_dir = Path(train_dir)
    image_list = list(data_dir.glob('*.jpeg'))
    n_images = len(image_list)
    
    if n_images == 9:
        plt.figure(figsize=(10, 10))
        for i in range(n_images):
            img = load_img(image_list[i]) 
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(img)
            plt.axis("off")
        plt.show()
        print("Check "+train_dir+(" for results (and remove them if you don't need them anymore)."))
    else:
        print("Error: Too many images in folder. Please delete old ones first.")

## 5. Load the image data for training and validation

Next step is to load our image data with the ImageDataGenerators that we defined before:

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(x, y),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=(x, y),
        batch_size=batch_size,
        class_mode='binary')

## 6. Train the CNN model

Now that we have everything set up we can start the model training of our defined CNN. The variables batch size, number of epochs, number of training and validation images have been defined at the very beginning. Have a quick look if everythings fine before you start the training. Training and validation progress is saved in 'history'.

In [ ]:
history = model.fit(
        train_generator,
        steps_per_epoch=nr_train_img // batch_size,
        epochs=nr_epoch,
        validation_data=validation_generator,
        validation_steps=nr_val_img // batch_size)

## 7. Visualize what your computer just did for you

We plot now the training and validation data for accuracy and loss during the model training. The output from above is further formatted using pandas so that you can go ahead and use it in your favorite program for plotting data.


Ideally you would want to see something like this:

![train_val](train_val.png)

In [ ]:
import matplotlib.pyplot as plt
import numpy

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(nr_epoch)

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

import pandas as pd

df = pd.DataFrame(history.history)

print(df)

## 8. Evaluating your model

Now that you have trained a model it's time to evaluate it. We use the eval data that you have prepared next to the training and validation data. For that we load the data from the directory the same way we loaded the validation data.


Make sure to pass a directory to eval_dir.

In [9]:
eval_dir = 'eval_nagarjun/'
evaluate_datagen = ImageDataGenerator(rescale=1./255)
evaluation_generator = evaluate_datagen.flow_from_directory(
        eval_dir,
        target_size=(x, y),
        batch_size=batch_size,
        class_mode='binary')

Found 4960 images belonging to 2 classes.


And we run the evaluate() method.

In [10]:
model.evaluate(evaluation_generator, return_dict = True)

310/310 [==============================] - 13s 41ms/step - loss: 0.2062 - accuracy: 0.9506


{'loss': 0.2062116414308548, 'accuracy': 0.9506048560142517}

Success! You went through all steps for training a model. The steps up to here have to be repeated several times until you figure out your model with the best fit for your data. Don't be discouraged if it takes a while!


If you have a good fit you can continue to save the model and its weights.

## 9. Save your data

Now you have seen the training/validation accuracy and loss graph and tested your new model with the evaluation data. If the values are nice and shiny it's time to save them. Adjust your naming accordingly. I tried to fit all important parameters into the naming for you to recognize it later again.

In [ ]:
train_dir_path = Path(train_dir)
data_dir_path = train_dir_path.parent
data_dir_name = data_dir_path.name

weights_file = PurePath(data_dir_path, data_dir_name + "_model" + str(dropout) + "_lr" + str(learning_rate) + "_ep" + str(nr_epoch) + "_weights.h5")
print(weights_file)
model_file = PurePath(data_dir_path, data_dir_name + "_model" + str(dropout) + "_lr" + str(learning_rate) + "_ep" + str(nr_epoch) + ".h5")
print(model_file)

In [ ]:
model.save_weights(weights_file)
model.save(model_file)